# Test the pipeline with the Hungarian Kalman Filter (WKF).

In [1]:
import os
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt

# Get the path of the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# Add the project root to sys.path if it is not already in sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the ImageSequenceDataset and create_dataloader from the pipeline module
from seismogram_extraction.pipeline import evaluate_filter, create_dataloader
from seismogram_extraction.filters.hungarian_kalman_filter import HungarianKalmanFilter

import torch.multiprocessing as mp

In [2]:
mp.set_start_method('spawn')  # or 'forkserver' if available

### Parameters
Dts = [0.005] #np.linspace(0.005, 1, 1) # Time step related to the state transition matrix A, ! different than sampling rate dt of signal s

sigma_ps = [1] # np.linspace(1e-2, 2, 1) 
sigma_vs = [0.8] # np.linspace(1e-2, 2, 1)
# sigma_a = np.linspace(1e-2, 2, 1)

P_vs = [10] # np.linspace(1e-2, 10, 1) # Initial state covariance
max_gaps = [10]
###

image_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines\overlap_0-28\signals"
GTs_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines\overlap_0-28\ground_truth"
    
dataloader = create_dataloader(image_folder_path, GTs_folder_path, batch_size=4)

output_folder_path = r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines\overlap_0-28\processed_HKF"
os.makedirs(output_folder_path, exist_ok=True)

best_parameters = None
min_MSE = float('inf')

for Dt in Dts:
    for sigma_p in sigma_ps:
        for sigma_v in sigma_vs:
            for P_v in P_vs:
                for max_gap in max_gaps:

                    A = np.array([[1, Dt],
                                [0, 1]]).astype(np.float64)
                                            
                    H = np.array([[1, 0]]).astype(np.float64)

                    Q = np.array([[sigma_p**2, 0],
                                [0, sigma_v**2]])


                    # Initial state covariance given all_x_0
                    P = np.zeros((2, 2))
                    P[1, 1] = P_v

                    print(f"Parameters: Dt={Dt}, sigma_p={sigma_p}, sigma_v={sigma_v}, P_v={P_v}, max_gap={max_gap}")
                    MSE1, MSE2, MSE_std = evaluate_filter(image_folder_path, GTs_folder_path, output_folder_path, HungarianKalmanFilter(A, H, Q), save=True)
                    print()

                    if MSE1 - MSE2 != 0: print(f"MSE1={MSE1}, MSE2={MSE2}, MSE_std={MSE_std}")
                    if MSE1 < min_MSE:
                        min_MSE = MSE1
                        best_parameters = (Dt, sigma_p, sigma_v, P_v, max_gap)

print(f"Best parameters: {best_parameters}, MSE={min_MSE}")

Parameters: Dt=0.005, sigma_p=1, sigma_v=0.8, P_v=10, max_gap=10


Batches:  33%|███▎      | 1/3 [00:12<00:24, 12.43s/it]


MSE of the batch 0: 933.5458415341822


Batches:  67%|██████▋   | 2/3 [00:17<00:07,  7.92s/it]


MSE of the batch 1: 2274.075027779619


Batches: 100%|██████████| 3/3 [00:19<00:00,  5.35s/it]


MSE of the batch 2: 1827.4806147046202


Batches: 100%|██████████| 3/3 [00:20<00:00,  6.88s/it]

Total Average MSE: 1678.3671613394738
Std: 557.3333779554347

Best parameters: (0.005, 1, 0.8, 10, 10), MSE=1678.3671613394738
